In [ ]:
# @title Import library (step 01)
from google.colab import files
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
import time

In [ ]:
# @title Load dataset (step 02)
!wget link_dataset \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

In [ ]:
# @title Extract dataset (step 03)
import os
import zipfile
# Ekstrak data file zip
local_zip = '/content/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/rockpaperscissors')
zip_ref.close()

In [ ]:
# @title Direktori dataset (step 04)
os.listdir('/content/rockpaperscissors/rockpaperscissors')
['rps-cv-images', 'scissors', 'rock', 'README_rpc-cv-images.txt', 'paper']

In [ ]:
# @title Path Target Directory (step 05)
base_dir = '/content/rockpaperscissors/rockpaperscissors/rps-cv-images'

In [ ]:
# @title gambar training proses (step 06)
paper = os.path.join('/content/rockpaperscissors/rockpaperscissors/paper')
rock = os.path.join('/content/rockpaperscissors/rockpaperscissors/rock')
scissors = os.path.join('/content/rockpaperscissors/rockpaperscissors/scissors')

In [ ]:
# @title list directory (step 07)
train_paper = os.listdir(paper)
train_rock = os.listdir(rock)
train_scissors = os.listdir(scissors)

print('Total training paper images:', len(train_paper))
print('Total training paper rock:', len(train_rock))
print('Total training scissors:', len(train_scissors))

In [ ]:
# @title datagenerator keras ke data augmentation (step 08)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# @title skala dan posisi gambar (step 09)
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20,
                                   horizontal_flip=True, shear_range=0.2,
                                   fill_mode='wrap', validation_split=0.4)
train_datagen

In [ ]:
# @title flow generate augemented dan ukuran pixel (step 10)
train_gen = train_datagen.flow_from_directory(base_dir, target_size=(100,150),
                                              shuffle=True,
                                              class_mode='categorical', subset='training')
validation_gen = train_datagen.flow_from_directory(base_dir, target_size=(100,150),
                                                   shuffle=True,
                                                   class_mode='categorical', subset='validation')

In [ ]:
# @title Tampilak sampel (step 11)
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nrows = 5
ncols = 5

pic_index = 0

fig = plt.gcf ()
fig.set_size_inches(ncols * 5, nrows * 5)

pic_index += 8
next_paper_pix = [os.path.join(paper, fname)
                for fname in train_paper[pic_index-8:pic_index]]
next_rock_pix = [os.path.join(rock, fname)
                for fname in train_rock[pic_index-8:pic_index]]
next_scissors_pix = [os.path.join(scissors, fname)
                for fname in train_scissors[pic_index-8:pic_index]]

for i, img_path in enumerate(next_paper_pix+next_rock_pix+next_scissors_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('off')
  img = mpimg.imread(img_path)
  plt.imshow(img)

In [ ]:
# @title model CNN (step 12)
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(100,150,3)),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
          tf.keras.layers.MaxPooling2D(2,2),
          tf.keras.layers.Dense (512, activation='relu'),
          tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# @title ringkasan model (step 13)
model.summary()

In [ ]:
# @title compile model (step 14)
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
# @title limit stop 98% (step 15)
accuracy_threshold = 98e-2
class my_callbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs.get('accuracy') >= accuracy_threshold:
      print('\nFor Epoch', epoch,
            '\nAccuracy has reach %2.2f%%' %(logs['accuracy']*100),
            ', training has been stopped')
      self.model.stop_training = True

In [ ]:
# @title Default title text
history = model.fit(train_gen, steps_per_epoch=25, epochs=20,
                    validation_data=validation_gen,
                    validation_steps=5, verbose=2,
                    callbacks=[my_callbacks()])

In [ ]:
# @title Default title text
plt.plot(history.history['loss'],label='Train')
plt.plot(history.history['val_loss'],label='Validation')
plt.legend()

In [ ]:
# @title Default title text
import numpy as np
from google.colab import files
from keras.preprocessing import image
uploaded = files.upload()
for fn in uploaded.keys():

   path = fn
   img = image.load_img(path, target_size=(100, 150))
   x = image.img_to_array(img)
   x = np.expand_dims(x, axis=0)

   images = np.vstack([x])
   classes = model.predict(images, batch_size=10)
   plt.axis('off')
   plt.imshow(img)
   plt.show()

   print(fn)
   if classes[0,0]==1:
     print('Image Classifier: Rock')
   elif classes[0,1]==1:
     print('Image Classifier: Paper')
   elif classes[0,2]==1:
     print('Image Classifier: Scissor')
   else:
      print('Image Classifier: None')